# Imports

In [ ]:
import API_functions.file_single as fs
import API_functions.file_batch as fb
import API_functions.file_info as fi
import API_functions.pre_process as pp
import API_functions.threshold_position_independent as tpi
import API_functions.dying as dy

In [ ]:
import os
import numpy as np

# A quick check about the method

In [ ]:
# Read image
single_path = "e:/3.Experimental_Data/Soil.column.2/0201/Reconstruct.Rename/"
file_name = "0103_rec00003312.bmp"
image = fs.read_image_opencv(str(os.path.join(single_path, file_name)))
fs.show_image(image, 4, (2, 2, 1))

# Get image info
fi.get_info(image)
hist = fi.calculate_hist(image)
fi.plot_hist(hist, 4, (2, 2, 2))

# Pre-process image
image_prepropossed = pp.median(image, kernel_size=3)
fs.show_image(image_prepropossed, 4, (2, 2, 3))

# Threshold image
image_threshold = tpi.user_threshold(image_prepropossed, 145)
# image_threshold = tpi.kmeans_3d(image_prepropossed)
fs.show_image(image_threshold, 4, (2, 2, 4))

# Work with the method above

## 3.1 Get Image Lists

In [ ]:
name_to_read = fb.ImageName(prefix="1-5_rec00", suffix="")
path_to_read = "e:/3.Experimental_Data/Soil.column.2/0205/2.Reconstruct/"
list_to_read = fb.get_image_names(path_to_read, name_to_read, "bmp")

## 3.2 ROI select

In [ ]:
roi = fb.roi_region(504, 666, 800, 800, z1=200, z2=3451)
path_roi_selected = "e:/3.Experimental_Data/Soil.column.2/0205/3.ROI/"
list_roi_selected = fb.roi_select(list_to_read, path=path_roi_selected, roi=roi)

## 3.3 Rename

In [ ]:
name_renamed = fb.ImageName(prefix="002_ou_DongYing", suffix="roi_selected")
path_renamed = "e:/3.Experimental_Data/Soil.column.2/0205/4.Rename/"
startid = 12261

# If the previous step meet an error
# list_roi_selected = fb.get_image_names("e:/3.Experimental_Data/Soil.column.2/*/*/", my_image_names=None, image_format="png")
list_roi_selected.reverse()
list_renamed = fb.rename(list_roi_selected, new_path = path_renamed, new_name=name_renamed, start_index=startid, overwrite=True)

## 3.4 Pre-process and Get its Threshold

In [ ]:
import cv2
import os
from tqdm import tqdm

In [ ]:
def image_process(namelists: list, save_path: str, save_name: fb.ImageName, start_index: int = 1):
    """
    Only for gray scale image.
    """
    
    temp_list = []

    for name in tqdm(namelists):
        image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        # image = fs.read_image_opencv(str(os.path.join(batch_read_path, name)))
        image_prepropossed = pp.median(image, 5)
        image_threshold = tpi.user_threshold(image_prepropossed, 145)
        image_invert = cv2.bitwise_not(image_threshold)     # invert image, make the pore space to be white

        save = os.path.join(save_path, f'{save_name.prefix}{str(start_index).zfill(5)}{save_name.suffix}.png')
        start_index += 1
        
        cv2.imwrite(save, image_invert)
        temp_list.append(save)

    fb.show_image_names(temp_list)
    return temp_list

In [ ]:
# If the previous step meet an error
# list_renamed = fb.get_image_names("e:/3.Experimental_Data/Soil.column.2/*/*/", my_image_names=None, image_format="png")

name_processed = fb.ImageName(prefix="002_ou_DongYing", suffix="processed")
path_processed = "e:/3.Experimental_Data/Soil.column.2/0205/5.Threshold/"
list_processed = image_process(list_renamed, save_name=name_processed, save_path=path_processed, start_index=startid)

## 3.5 Get nii File

In [ ]:
name_nii = "soil_column_2"
listPart1 = fb.get_image_names("e:/3.Experimental_Data/Soil.column.2/0201/5.Threshold/", my_image_names=None, image_format="png")
# listPart2 = fb.get_image_names("e:/3.Experimental_Data/Soil.column.2/0202/5.Threshold/", my_image_names=None, image_format="png")
# listPart3 = fb.get_image_names("e:/3.Experimental_Data/Soil.column.2/0203/5.Threshold/", my_image_names=None, image_format="png")
# listPart4 = fb.get_image_names("e:/3.Experimental_Data/Soil.column.2/0204/5.Threshold/", my_image_names=None, image_format="png")
# listPart5 = fb.get_image_names("e:/3.Experimental_Data/Soil.column.2/0205/5.Threshold/", my_image_names=None, image_format="png")

# combine_list = listPart1 + listPart2 + listPart3 + listPart4 + listPart5


In [ ]:
for i in range(1, 6):
    if i == 1:
        path_nii = "e:/3.Experimental_Data/Soil.column.2/"
        listPart = eval(f"listPart{i}")
        name_nii = f"soil_column_2__{i}"
        painted_volume, connection_part_list = dy.dying_color(listPart, 20, 5)
        fb.create_nifti(painted_volume, path_nii, name_nii)

In [ ]:
# import numpy as np
# np.sum(painted_volume)
# temp = painted_volume[:,:,51]
connection_part_list